## Manage Dataproc Workflows using gcloud Commands
Let us see how we can manage Dataproc Workflows using gcloud commands.
* Step 1: Create Dataproc Workflow Template
* Step 2: Configure active Dataproc cluster (we can also configure new cluster)
* Step 3: Add Spark SQL or Pyspark Jobs to Dataproc Workflow Templates with Dependencies
* Step 4: Run and Validate the Dataproc Workflow Template

We can take care of all the steps using `gcloud` commands.

In [1]:
!gcloud config set dataproc/region us-central1

Updated property [dataproc/region].


Updates are available for some Google Cloud CLI components.  To install them,
please run:
  $ gcloud components update



In [2]:
!gcloud dataproc workflow-templates

ERROR: (gcloud.dataproc.workflow-templates) Command name argument expected.

Available groups for gcloud dataproc workflow-templates:

      add-job                 Add Dataproc jobs to workflow template.

Available commands for gcloud dataproc workflow-templates:

      create                  Create a workflow template.
      delete                  Delete a workflow template.
      describe                Describe a workflow template.
      export                  Export a workflow template.
      get-iam-policy          Get IAM policy for a workflow template.
      import                  Import a workflow template.
      instantiate             Instantiate a workflow template.
      instantiate-from-file   Instantiate a workflow template from a file.
      list                    List workflow templates.
      remove-dag-timeout      Remove DAG timeout from a workflow template.
      remove-job              Remove a job from workflow template.
      set-cluster-selector    Set clu

In [3]:
!gcloud dataproc workflow-templates list

ID               JOBS  UPDATE_TIME                  VERSION
getting-started  4     2023-06-11T23:01:29.613712Z  1


Here is the command to delete Dataproc Workflow Template (multiline approach doesn't work on Windows)

```shell
gcloud dataproc workflow-templates \
    delete wf-daily-product-revenue
```

In [ ]:
!gcloud dataproc workflow-templates delete wf-daily-product-revenue --quiet

Here is the command to create Dataproc Workflow.

```shell
gcloud dataproc workflow-templates \
    create wf-daily-product-revenue
```

In [4]:
!gcloud dataproc workflow-templates create

ERROR: (gcloud.dataproc.workflow-templates.create) argument (TEMPLATE : --region=REGION): Must be specified.
Usage: gcloud dataproc workflow-templates create (TEMPLATE : --region=REGION) [optional flags]
  optional flags may be  --dag-timeout | --help | --labels | --region

For detailed information on this command and its flags, run:
  gcloud dataproc workflow-templates create --help


In [5]:
!gcloud dataproc workflow-templates create wf-daily-product-revenue

In [6]:
!gcloud dataproc workflow-templates list

ID                        JOBS  UPDATE_TIME                  VERSION
getting-started           4     2023-06-11T23:01:29.613712Z  1
wf-daily-product-revenue  0     2023-06-12T00:08:35.526710Z  1


In [7]:
!gcloud dataproc workflow-templates 

ERROR: (gcloud.dataproc.workflow-templates) Command name argument expected.

Available groups for gcloud dataproc workflow-templates:

      add-job                 Add Dataproc jobs to workflow template.

Available commands for gcloud dataproc workflow-templates:

      create                  Create a workflow template.
      delete                  Delete a workflow template.
      describe                Describe a workflow template.
      export                  Export a workflow template.
      get-iam-policy          Get IAM policy for a workflow template.
      import                  Import a workflow template.
      instantiate             Instantiate a workflow template.
      instantiate-from-file   Instantiate a workflow template from a file.
      list                    List workflow templates.
      remove-dag-timeout      Remove DAG timeout from a workflow template.
      remove-job              Remove a job from workflow template.
      set-cluster-selector    Set clu

In [8]:
!gcloud dataproc workflow-templates set-cluster-selector

ERROR: (gcloud.dataproc.workflow-templates.set-cluster-selector) argument (TEMPLATE : --region=REGION): Must be specified.
Usage: gcloud dataproc workflow-templates set-cluster-selector (TEMPLATE : --region=REGION) [optional flags]
  optional flags may be  --cluster-labels | --help | --region

For detailed information on this command and its flags, run:
  gcloud dataproc workflow-templates set-cluster-selector --help


Here is the command to attach running or active Dataproc Cluster to the Dataproc Workflow. We need to specify the label for the cluster.

```shell
gcloud dataproc workflow-templates \
    set-cluster-selector \
    wf-daily-product-revenue \
    --cluster-labels goog-dataproc-cluster-name=aidataprocdev
```

In [9]:
!gcloud dataproc workflow-templates set-cluster-selector wf-daily-product-revenue \
     --cluster-labels goog-dataproc-cluster-name='cluster-256f'

In [ ]:
!gcloud dataproc workflow-templates add-job

In [ ]:
!gcloud dataproc workflow-templates add-job spark-sql

* The command `gcloud dataproc workflow-templates add-job` is similar to `gcloud dataproc jobs submit`. Here are the examples for submitting jobs using `gcloud dataproc jobs submit`.

```shell
# Without parameters
gcloud dataproc jobs submit \
    spark-sql --cluster=aidataprocdev \
    -f gs://airetail/scripts/daily_product_revenue/cleanup.sql

# With parameters
gcloud dataproc jobs submit \
    spark-sql --cluster=aidataprocdev \
    -f gs://airetail/scripts/daily_product_revenue/file_format_converter.sql \
    --params=bucket_name=gs://airetail,table_name=orders
```


Here are the commands to add Spark SQL Jobs to the Dataproc Workflow.

```shell
gcloud dataproc workflow-templates add-job spark-sql \
    --step-id=job-cleanup \
    --file=gs://airetail/scripts/daily_product_revenue/cleanup.sql \
    --workflow-template=wf-daily-product-revenue

# File Format Converter jobs with dependency on cleanup
gcloud dataproc workflow-templates add-job spark-sql \
    --step-id=job-convert-orders \
    --file=gs://airetail/scripts/daily_product_revenue/file_format_converter.sql \
    --params=bucket_name=gs://airetail,table_name=orders \
    --workflow-template=wf-daily-product-revenue \
    --start-after=job-cleanup

gcloud dataproc workflow-templates add-job spark-sql \
    --step-id=job-convert-order-items \
    --file=gs://airetail/scripts/daily_product_revenue/file_format_converter.sql \
    --params=bucket_name=gs://airetail,table_name=order_items \
    --workflow-template=wf-daily-product-revenue \
    --start-after=job-cleanup

# Last Job which depends on convert orders and order_items jobs
gcloud dataproc workflow-templates add-job spark-sql \
    --step-id=job-daily-product-revenue \
    --file=gs://airetail/scripts/daily_product_revenue/compute_daily_product_revenue.sql \
    --params=bucket_name=gs://airetail \
    --workflow-template=wf-daily-product-revenue \
    --start-after=job-convert-orders,job-convert-order-items
```

In [10]:
!gcloud dataproc workflow-templates add-job spark-sql --step-id=job-cleanup \
     --file=gs://airetail_mld/scripts/daily_product_revenue/cleanup.sql \ 
     --workflow-template=wf-daily-product-revenue

createTime: '2023-06-12T00:08:35.526710Z'
id: wf-daily-product-revenue
jobs:
- sparkSqlJob:
    queryFileUri: gs://airetail_mld/scripts/daily_product_revenue/cleanup.sql
  stepId: job-cleanup
name: projects/dataanalytics-347914/regions/us-central1/workflowTemplates/wf-daily-product-revenue
placement:
  clusterSelector:
    clusterLabels:
      goog-dataproc-cluster-name: cluster-256f
updateTime: '2023-06-12T00:11:59.876747Z'
version: 3


In [17]:

!gcloud dataproc workflow-templates add-job spark-sql --step-id=job-convert-orders \
     --file=gs://airetail_mld/scripts/daily_product_revenue/file_format_converter.sql \
     --params=bucket_name=gs://airetail_mld,table_name=orders \
     --workflow-template=wf-daily-product-revenue \
     --start-after=job-cleanup

createTime: '2023-06-12T00:08:35.526710Z'
id: wf-daily-product-revenue
jobs:
- sparkSqlJob:
    queryFileUri: gs://airetail_mld/scripts/daily_product_revenue/cleanup.sql
  stepId: job-cleanup
- prerequisiteStepIds:
  - job-cleanup
  sparkSqlJob:
    queryFileUri: gs://airetail_mld/scripts/daily_product_revenue/file_format_converter.sql
    scriptVariables:
      bucket_name: gs://airetail_mld
      table_name: order_items
  stepId: job-convert-order-items
- prerequisiteStepIds:
  - job-cleanup
  sparkSqlJob:
    queryFileUri: gs://airetail_mld/scripts/daily_product_revenue/file_format_converter.sql
    scriptVariables:
      bucket_name: gs://airetail_mld
      table_name: orders
  stepId: job-convert-orders
name: projects/dataanalytics-347914/regions/us-central1/workflowTemplates/wf-daily-product-revenue
placement:
  clusterSelector:
    clusterLabels:
      goog-dataproc-cluster-name: cluster-256f
updateTime: '2023-06-12T00:16:08.723102Z'
version: 5


In [15]:
!gcloud dataproc workflow-templates add-job spark-sql --step-id=job-convert-order-items \
    --file=gs://airetail_mld/scripts/daily_product_revenue/file_format_converter.sql \
    --params=bucket_name=gs://airetail_mld,table_name=order_items \
    --workflow-template=wf-daily-product-revenue --start-after=job-cleanup

createTime: '2023-06-12T00:08:35.526710Z'
id: wf-daily-product-revenue
jobs:
- sparkSqlJob:
    queryFileUri: gs://airetail_mld/scripts/daily_product_revenue/cleanup.sql
  stepId: job-cleanup
- prerequisiteStepIds:
  - job-cleanup
  sparkSqlJob:
    queryFileUri: gs://airetail_mld/scripts/daily_product_revenue/file_format_converter.sql
    scriptVariables:
      bucket_name: gs://airetail_mld
      table_name: order_items
  stepId: job-convert-order-items
name: projects/dataanalytics-347914/regions/us-central1/workflowTemplates/wf-daily-product-revenue
placement:
  clusterSelector:
    clusterLabels:
      goog-dataproc-cluster-name: cluster-256f
updateTime: '2023-06-12T00:15:51.731392Z'
version: 4


In [18]:
!gcloud dataproc workflow-templates add-job spark-sql \
    --step-id=job-daily-product-revenue \
    --file=gs://airetail_mld/scripts/daily_product_revenue/compute_daily_product_revenue.sql \
    --params=bucket_name=gs://airetail_mld \
    --workflow-template=wf-daily-product-revenue \
    --start-after=job-convert-orders,job-convert-order-items

createTime: '2023-06-12T00:08:35.526710Z'
id: wf-daily-product-revenue
jobs:
- sparkSqlJob:
    queryFileUri: gs://airetail_mld/scripts/daily_product_revenue/cleanup.sql
  stepId: job-cleanup
- prerequisiteStepIds:
  - job-cleanup
  sparkSqlJob:
    queryFileUri: gs://airetail_mld/scripts/daily_product_revenue/file_format_converter.sql
    scriptVariables:
      bucket_name: gs://airetail_mld
      table_name: order_items
  stepId: job-convert-order-items
- prerequisiteStepIds:
  - job-cleanup
  sparkSqlJob:
    queryFileUri: gs://airetail_mld/scripts/daily_product_revenue/file_format_converter.sql
    scriptVariables:
      bucket_name: gs://airetail_mld
      table_name: orders
  stepId: job-convert-orders
- prerequisiteStepIds:
  - job-convert-orders
  - job-convert-order-items
  sparkSqlJob:
    queryFileUri: gs://airetail_mld/scripts/daily_product_revenue/compute_daily_product_revenue.sql
    scriptVariables:
      bucket_name: gs://airetail_mld
  stepId: job-daily-product-revenue

In [19]:
!gcloud dataproc workflow-templates list

ID                        JOBS  UPDATE_TIME                  VERSION
getting-started           4     2023-06-11T23:01:29.613712Z  1
wf-daily-product-revenue  4     2023-06-12T00:18:39.027622Z  6


In [20]:
!gcloud dataproc workflow-templates describe wf-daily-product-revenue

createTime: '2023-06-12T00:08:35.526710Z'
id: wf-daily-product-revenue
jobs:
- sparkSqlJob:
    queryFileUri: gs://airetail_mld/scripts/daily_product_revenue/cleanup.sql
  stepId: job-cleanup
- prerequisiteStepIds:
  - job-cleanup
  sparkSqlJob:
    queryFileUri: gs://airetail_mld/scripts/daily_product_revenue/file_format_converter.sql
    scriptVariables:
      bucket_name: gs://airetail_mld
      table_name: order_items
  stepId: job-convert-order-items
- prerequisiteStepIds:
  - job-cleanup
  sparkSqlJob:
    queryFileUri: gs://airetail_mld/scripts/daily_product_revenue/file_format_converter.sql
    scriptVariables:
      bucket_name: gs://airetail_mld
      table_name: orders
  stepId: job-convert-orders
- prerequisiteStepIds:
  - job-convert-orders
  - job-convert-order-items
  sparkSqlJob:
    queryFileUri: gs://airetail_mld/scripts/daily_product_revenue/compute_daily_product_revenue.sql
    scriptVariables:
      bucket_name: gs://airetail_mld
  stepId: job-daily-product-revenue

Here is the command to instantiate or run Dataproc Workflow.

```shell
gcloud dataproc workflow-templates \
    instantiate wf-daily-product-revenue
```

In [21]:
!gcloud dataproc workflow-templates

ERROR: (gcloud.dataproc.workflow-templates) Command name argument expected.

Available groups for gcloud dataproc workflow-templates:

      add-job                 Add Dataproc jobs to workflow template.

Available commands for gcloud dataproc workflow-templates:

      create                  Create a workflow template.
      delete                  Delete a workflow template.
      describe                Describe a workflow template.
      export                  Export a workflow template.
      get-iam-policy          Get IAM policy for a workflow template.
      import                  Import a workflow template.
      instantiate             Instantiate a workflow template.
      instantiate-from-file   Instantiate a workflow template from a file.
      list                    List workflow templates.
      remove-dag-timeout      Remove DAG timeout from a workflow template.
      remove-job              Remove a job from workflow template.
      set-cluster-selector    Set clu

In [22]:
!gcloud dataproc workflow-templates instantiate

ERROR: (gcloud.dataproc.workflow-templates.instantiate) argument (TEMPLATE : --region=REGION): Must be specified.
Usage: gcloud dataproc workflow-templates instantiate (TEMPLATE : --region=REGION) [optional flags]
  optional flags may be  --async | --help | --parameters | --region

For detailed information on this command and its flags, run:
  gcloud dataproc workflow-templates instantiate --help


In [23]:
!gcloud dataproc workflow-templates instantiate-from-file

ERROR: (gcloud.dataproc.workflow-templates.instantiate-from-file) argument --file: Must be specified.
Usage: gcloud dataproc workflow-templates instantiate-from-file --file=FILE [optional flags]
  optional flags may be  --async | --help | --region

For detailed information on this command and its flags, run:
  gcloud dataproc workflow-templates instantiate-from-file --help


In [24]:
!gcloud dataproc workflow-templates instantiate-from-file --help

NAME
    gcloud dataproc workflow-templates instantiate-from-file - instantiate a
        workflow template from a file

SYNOPSIS
    gcloud dataproc workflow-templates instantiate-from-file --file=FILE
        [--async] [--region=REGION] [GCLOUD_WIDE_FLAG ...]

DESCRIPTION
    Instantiate a workflow template from a file.

EXAMPLES
    To instantiate a workflow template from a yaml file 'template.yaml' in
    region 'us-central1', run:

        $ gcloud dataproc workflow-templates instantiate-from-file \
            --file=template.yaml --region=us-central1

REQUIRED FLAGS
     --file=FILE
        The YAML file containing the workflow template to run

OPTIONAL FLAGS
     --async
        Return immediately, without waiting for the operation in progress to
        complete.

     --region=REGION
        Dataproc region to use. Each Dataproc region constitutes an independent
        resource namespace constrained to deploying instances into Compute
        Engine zones inside the region. 

In [25]:
!gcloud dataproc workflow-templates export

ERROR: (gcloud.dataproc.workflow-templates.export) argument (TEMPLATE : --region=REGION): Must be specified.
Usage: gcloud dataproc workflow-templates export (TEMPLATE : --region=REGION) [optional flags]
  optional flags may be  --destination | --help | --region | --version

For detailed information on this command and its flags, run:
  gcloud dataproc workflow-templates export --help


In [26]:
!gcloud dataproc workflow-templates export wf-daily-product-revenue

jobs:
- sparkSqlJob:
    queryFileUri: gs://airetail_mld/scripts/daily_product_revenue/cleanup.sql
  stepId: job-cleanup
- prerequisiteStepIds:
  - job-cleanup
  sparkSqlJob:
    queryFileUri: gs://airetail_mld/scripts/daily_product_revenue/file_format_converter.sql
    scriptVariables:
      bucket_name: gs://airetail_mld
      table_name: order_items
  stepId: job-convert-order-items
- prerequisiteStepIds:
  - job-cleanup
  sparkSqlJob:
    queryFileUri: gs://airetail_mld/scripts/daily_product_revenue/file_format_converter.sql
    scriptVariables:
      bucket_name: gs://airetail_mld
      table_name: orders
  stepId: job-convert-orders
- prerequisiteStepIds:
  - job-convert-orders
  - job-convert-order-items
  sparkSqlJob:
    queryFileUri: gs://airetail_mld/scripts/daily_product_revenue/compute_daily_product_revenue.sql
    scriptVariables:
      bucket_name: gs://airetail_mld
  stepId: job-daily-product-revenue
placement:
  clusterSelector:
    clusterLabels:
      goog-dataproc-c

In [ ]:
# This will take some time to run
!gcloud dataproc workflow-templates instantiate wf-daily-product-revenue